In [7]:
import pandas as pd
import urllib.request
import uuid
from PIL import Image
import requests
from io import BytesIO

import data_collector

## Read in image url data

I'm using the google open image dataset v4, you can download the aggregated image info from here: https://storage.googleapis.com/openimages/web/download.html .  
I need more file:
    * The file which contains the ImageID and the urls where we can download the real images, this will be the image_id_url DataFrame. 
    * A class description file, which is a switch table between Image label names and their identifiers.  
    * The image label file, where ImageID and LabelId connected. It is separated in each dataset (train,valid,test).  
First load up into the memory the csv-s.


In [ ]:
image_id_url = pd.read_csv('O:\\ProgrammingSoftwares\\anaconda_projects\\dp_nagyhazi\\data\\image_ids_and_rotation.csv',engine='python',sep = ',')

In [ ]:
image_id_url.head()

In [ ]:
image_labels = pd.read_csv('O:\\ProgrammingSoftwares\\anaconda_projects\\dp_nagyhazi\\data\\train-annotations-human-imagelabels.csv',engine='python',sep = ',')

In [ ]:
image_validation_labels = pd.read_csv('O:\\ProgrammingSoftwares\\anaconda_projects\\dp_nagyhazi\\data\\validation-annotations-human-imagelabels.csv',engine='python',sep = ',')
image_test_labels = pd.read_csv('O:\\ProgrammingSoftwares\\anaconda_projects\\dp_nagyhazi\\data\\test-annotations-human-imagelabels.csv',engine='python',sep = ',')

Summarize the data. From the website you can download one-by-one, but i don't need it separately. 

In [ ]:
image_labels = pd.concat( [image_labels,image_validation_labels, image_test_labels],ignore_index=True )
image_labels.head()

In [ ]:
class_description = pd.read_csv('O:\\ProgrammingSoftwares\\anaconda_projects\\dp_nagyhazi\\data\\class-descriptions.csv',engine='python',header = None, sep = ',')

In [ ]:
class_description.head()

I want to choose those files which contains cities. I find its label id and save into (city_label).

In [ ]:
city_label = class_description[class_description[1] == 'Skyline'][0]
city_label = city_label.values[0]

After that i collect all image id, what label equal to city_label. Confidence is important beause there are pictures in the dataset which label is city, but doesn't contains one.

In [ ]:
city_id_df = image_labels.loc[(image_labels['LabelName'] == city_label) & (image_labels['Confidence'] == 1)]

I inner join the collected city_id_df with the biggest table: image_id_url (this contains urls and author informations) on ImageID.

In [ ]:
merged_df = image_id_url.merge(city_id_df, on='ImageID', how='inner')

I would like to use the smallest size of the picture, i don't need the best quality now. I'm filter those rows where the smallest image url column is not empty.

In [ ]:
image_urls = merged_df.loc[pd.notnull(merged_df['Thumbnail300KURL'])]

Example data from our dataset.

In [ ]:
response = requests.get(image_urls['Thumbnail300KURL'][11])
img = Image.open(BytesIO(response.content))
img.show()

Check if are there any picture which not jpg

In [ ]:
not_jpg_df = image_urls.loc[image_urls['Thumbnail300KURL'].str.rsplit('.',1).apply(lambda x: 'jpg' not in x[1])]
list(not_jpg_df)

This is just the Dataframe header row, so there are only jpg url page. The bad reality is that flickr response back a few time an blank image whith "Image no longer exists" with HTTP status code 200, luckily its Content-Type is image/png so i can filter these pictures out.

In [4]:
image_folder = 'O:/ProgrammingSoftwares/anaconda_projects/dp_nagyhazi/data/images/'
data_handler = DataCollector()
data_handler.load_datas(image_id_rotation_csv, train_annotation_csv, validation_annotation_csv, test_annotation_csv, class_desc_csv)
#data_handler.collect_small_images(image_urls, image_folder)

In [5]:
data_handler.find_by_labelName('Skyline')

,ImageID,Subset,OriginalURL,OriginalLandingURL,License,AuthorProfileURL,Author,Title,OriginalSize,OriginalMD5,Thumbnail300KURL,Rotation,Source,LabelName,Confidence
0,102d0db6a504fb67,train,https://farm8.staticflickr.com/5503/9460579392...,https://www.flickr.com/photos/ch-villa/9460579392,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/ch-villa/,Nat Ch Villa,Untitled,2513214,M4maZw5avU95s+fPWpy5LA==,https://c7.staticflickr.com/6/5503/9460579392_...,0.0,crowdsource-verification,/m/0204fg,1
1,c8804fa0a1377b24,train,https://farm5.staticflickr.com/8329/8135587135...,https://www.flickr.com/photos/alecperkins/8135...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/alecperkins/,Alec Perkins,Lower Manhattan,130960,UEF/oGQZgi0v3VNIV75npw==,https://c4.staticflickr.com/9/8329/8135587135_...,0.0,verification,/m/0204fg,1
2,0029918fa1ddc5d7,train,https://farm8.staticflickr.com/5157/1396066884...,https://www.flickr.com/photos/black_white_pict...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/black_white_pict...,Historical images,Turkey Istanbul Galata port 1970's,305695,eX37cDyJLxECVVpg5cR6ag==,https://c1.staticflickr.com/6/5157/13960668847...,0.0,verification,/m/0204fg,1
3,0058ff85b0ffb9b9,train,https://c2.staticflickr.com/9/8513/8515718161_...,https://www.flickr.com/photos/chrisr3i/8515718161,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/chrisr3i/,Christian Russo,Edimburgo Dal Trono di ArtÃ¹,1459391,nEu3UNveEkPdoNdR/4fC3w==,https://c6.staticflickr.com/9/8513/8515718161_...,0.0,verification,/m/0204fg,1
4,0d818dc8c72ca618,train,https://c1.staticflickr.com/7/6132/6038035328_...,https://www.flickr.com/photos/robsandbach/6038...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/robsandbach/,rob.sandbach,Ho chi Minh / Saigon,426722,hz1Hf1dyYwA6hp8yMvJgtQ==,https://c2.staticflickr.com/7/6132/6038035328_...,0.0,crowdsource-verification,/m/0204fg,1
5,005df37133b140fc,train,https://c5.staticflickr.com/6/5181/5685420097_...,https://www.flickr.com/photos/51252776@N04/568...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/51252776@N04/,Colin Capelle,DC Skyline at Night,1791937,UuPOp2NrsiLv/HJBMI/dGw==,https://c7.staticflickr.com/6/5181/5685420097_...,0.0,verification,/m/0204fg,1
6,0025d1950652e2f8,train,https://c1.staticflickr.com/8/7020/6739450819_...,https://www.flickr.com/photos/daquellamanera/6...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/daquellamanera/,Daniel Lobo,Paseo de la Castellana,576901,7umWEIMPz8KoTG4uJmSs4w==,https://c4.staticflickr.com/8/7020/6739450819_...,0.0,verification,/m/0204fg,1
7,009a18444f7f3037,train,https://farm8.staticflickr.com/8059/8206214293...,https://www.flickr.com/photos/mostad/8206214293,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/mostad/,Robert Harley Mostad,IMG_2849,2860334,QZs1MKr2pFu7fB/Ap+ygRw==,https://c6.staticflickr.com/9/8059/8206214293_...,0.0,verification,/m/0204fg,1
8,dca95311d7b94a05,train,https://farm6.staticflickr.com/43/110539097_02...,https://www.flickr.com/photos/bootbearwdc/1105...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/bootbearwdc/,David,Untitled,188985,gXQAqSskStC/UaTeDvJoPA==,https://c4.staticflickr.com/1/43/110539097_027...,0.0,verification,/m/0204fg,1
9,0fd446e795f62ae5,train,https://farm6.staticflickr.com/2204/5794428048...,https://www.flickr.com/photos/gazeronly/579442...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/gazeronly/,torbakhopper,teeth whitening approved : downtown san franci...,931884,bhzK+NYWZ6dPoHezVyFU4A==,https://c3.staticflickr.com/3/2204/5794428048_...,0.0,crowdsource-verification,/m/0204fg,1


In [6]:
data_handler.collect_small_images(data_handler.result_label_df , image_folder)

In [3]:
image_id_rotation_csv = 'O:\\ProgrammingSoftwares\\anaconda_projects\\dp_nagyhazi\\data\\image_ids_and_rotation.csv'
train_annotation_csv = 'O:\\ProgrammingSoftwares\\anaconda_projects\\dp_nagyhazi\\data\\train-annotations-human-imagelabels.csv'
validation_annotation_csv = 'O:\\ProgrammingSoftwares\\anaconda_projects\\dp_nagyhazi\\data\\validation-annotations-human-imagelabels.csv'
test_annotation_csv = 'O:\\ProgrammingSoftwares\\anaconda_projects\\dp_nagyhazi\\data\\test-annotations-human-imagelabels.csv'
class_desc_csv = 'O:\\ProgrammingSoftwares\\anaconda_projects\\dp_nagyhazi\\data\\class-descriptions.csv'

class DataCollector:
    

    def load_datas(self, image_id_url_csv, train_annotation_csv, validation_annotation_csv, test_annotation_csv, class_desc_csv):
        self.image_id_url = pd.read_csv(image_id_url_csv,engine='python', sep = ',')

        self.image_train_labels = pd.read_csv(train_annotation_csv,engine='python', sep = ',')
        self.image_validation_labels = pd.read_csv(validation_annotation_csv, engine='python',sep = ',')
        self.image_test_labels = pd.read_csv(test_annotation_csv,engine='python',sep = ',')

        self.class_description = pd.read_csv(class_desc_csv,engine='python',header = None, sep = ',')

        self.image_labels = pd.concat( [self.image_train_labels,self.image_validation_labels, self.image_test_labels] )
    
    def find_by_labelName(self,label_name):
        label_id = self.class_description[self.class_description[1] == label_name][0]
        label_id = label_id.values[0]
        
        labeled_df = self.image_labels.loc[(self.image_labels['LabelName'] == label_id) & (self.image_labels['Confidence'] == 1)]
        merged_df = self.image_id_url.merge(labeled_df, on='ImageID', how='inner')
        
        self.result_label_df = merged_df.loc[pd.notnull(merged_df['Thumbnail300KURL'])]
        return self.result_label_df
        
    
    '''Long running iteration!!
    Iterate through row-by-row, download and save image. If the Http header Content-Type == image/png
    then it won't download the image. This is because blank image comes with "Image no longer exists", HTTP status code 200.
        Parameters: 
            - image_urls: Pandas DataFrame cleaned url links, skip if url link is None.
            - folder: The folder where do you want to download the images.
    '''
    def collect_small_images(self,image_urls,folder):
        for index, row in image_urls.iterrows():

            if not pd.notnull(row['Thumbnail300KURL']):
                return
            url = row['Thumbnail300KURL']
            filename = folder + str(uuid.uuid1())+ '.jpg'
            # Checking png is important, because there are pictures which is no longer exist, therefore im getting back a picture
            # with http status code 200 but it is a blank pic.
            if 'image/png' not in urllib.request.urlopen(url).info()['Content-Type']:
                urllib.request.urlretrieve(url, filename)